<a href="https://colab.research.google.com/github/tsunamizou/Hindawi/blob/main/nature_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from bs4 import BeautifulSoup
import requests
import csv

This scraper is to get all the affiliated institutions of the leading 500 institutions from China

In [4]:
link = "https://www.natureindex.com/supplements/nature-index-2022-big-5/tables/overall"
page = requests.get(link)
soup = BeautifulSoup (page.content, "html.parser")


In [5]:
tbody = soup.find('tbody')
rows = tbody.find_all('tr')



In [ ]:

url_list = []
cn_list = []
for row in rows:
  if row.find(class_="country").text == 'China':  
    url_link = row.find('a')
    full_link = 'https://www.natureindex.com' + url_link['href']
    cn_name = row.find('a').text.replace('\n','').strip()
    url_list.append(full_link)
    cn_list.append(cn_name)

print(url_list)
print(cn_list)
len(cn_list)

In [ ]:
cn_affli_list = []

# There could be two divs under the "Relationships" section, the first one starts with the institute name followed by a list of directly affilliated institutes, which is what we want.
# The other one starts with 'Affiliated joint institutions and consortia', which is not what we want

for test_link in url_list:
  detail = requests.get(test_link)
  soup_2 = BeautifulSoup(detail.content, "html.parser")

  try:
    ins_name = soup_2.find('h3',class_="u-sans-serif u-mb-8").text
  except AttributeError:
    ins_name = 'N/A'
  if ins_name == 'Affiliated joint institutions and consortia':
    ins_name = 'N/A'
  if ins_name == 'N/A':
    affli_list = 'N/A'
  else:
    affli_table = soup_2.find('ul', class_="font-weight-bold")
    try:
      affli_list = [ affli.find('a').text for affli in affli_table.find_all('li')]
    except AttributeError:
      affli_list='N/A'
  cn_affli_list.append(affli_list)

print(cn_affli_list)
len(cn_affli_list)

In [ ]:
# creating a dictionary with the keys being the institute name, and the values being a list of its affiliations

cn_dict = dict(zip(cn_list, cn_affli_list))
print(cn_dict)

In [26]:

new_path = open("China_institutes.csv", "w")

z = csv.writer(new_path)
for new_k, new_v in cn_dict.items():
    z.writerow([new_k, new_v])

new_path.close()

# open the csv file in excel and replace all the ["  ['  ']  "]  ","  ','  ",'  ',"
# with |, then split text into columns
# there should be a better way of doing this, if someone knows please tell me 